In [86]:
from metafid.deriv import Pricing
from metafid.deriv import OptionStrategy
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import re 
import finpy_tse as fpy
import psycopg
import jdatetime as jdt
import numpy as np
import re
from CONFIG import DB_PASS as PASS

In [87]:
bs = Pricing()
now = jdt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
now

'1402-01-02 20:29:37'

In [88]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [89]:
drop_cols = ["isin","time","open","close","no","volume","low","high","y_final","eps","base_vol","unknown1","unknown2","sector","day_ul","day_ll", "share_no", "mkt_id", "sell_no", "buy_no"]
ua_list = ["شپنا",
"خودرو",
"فملي",
"شستا",
"وبصادر",
"فولاد",
"خگستر",
"سلام",
"دي",
"سرو",
"خاور",
"وبملت",
"خساپا",
"فخوز",
"فرابورس",
"اهرم",
"حسير",
"خپارس",
"هاي وب",
"چكاپا",
"توسن",
"وتجارت",
"ذوب",
"بساما",
"شبندر",
"سمگا",
"وخاور",
"شتران",
"كاريس",
"ت ثاميد",
"پالايش",
"لبخند",
"تپسي",
"كارا"]

def mw(drop_cols:list=None):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    r = requests.get(
        "http://www.tsetmc.com/tsev2/data/MarketWatchPlus.aspx", headers=headers
    )
    main_text = r.text
    ob_df = pd.DataFrame((main_text.split("@")[3]).split(";"))
    ob_df = ob_df[0].str.split(",", expand=True)
    ob_df.columns = [
        "web_id",
        "ob_depth",
        "sell_no",
        "buy_no",
        "buy_price",
        "sell_price",
        "buy_vol",
        "sell_vol",
    ]
    ob_df = ob_df[
        [
            "web_id",
            "ob_depth",
            "sell_no",
            "sell_vol",
            "sell_price",
            "buy_price",
            "buy_vol",
            "buy_no",
        ]
    ]
    ob_df.set_index("web_id", inplace=True)

    mw_df = pd.DataFrame((main_text.split("@")[2]).split(";"))
    mw_df = mw_df[0].str.split(",", expand=True)
    mw_df = mw_df.iloc[:, :23]
    mw_df.columns = [
        "web_id",
        "isin",
        "symbol",
        "name",
        "time",
        "open",
        "final",
        "close",
        "no",
        "volume",
        "value",
        "low",
        "high",
        "y_final",
        "eps",
        "base_vol",
        "unknown1",
        "unknown2",
        "sector",
        "day_ul",
        "day_ll",
        "share_no",
        "mkt_id",
    ]
    mw_df.set_index("web_id", inplace=True)
    df = mw_df.join(ob_df)
    df = df.assign(dt=jdt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    if drop_cols:
        return df.drop(columns = drop_cols, axis=1)
    else:
        return df

In [90]:
def option_data():
    mw_df = mw(drop_cols=drop_cols)

    ua_df = mw_df[(mw_df.symbol.isin(ua_list)) & (mw_df.value.astype(int) > 0) & (mw_df.ob_depth.astype(int)==1)].add_prefix("ua_")
    ua_df.rename(columns={"ua_symbol":"ua"}, inplace=True)
    ua_df.drop_duplicates(inplace=True)
    
    mw_df = mw_df[mw_df.name.str.startswith("اختيار")]
    def expand_option_info(df):
        def clean_date(x):
            date_ =re.findall("[0-9]+", x)
            date_ = "".join(date_)
            if len(date_)==8:
                return f"{date_[:4]}-{date_[4:6]}-{date_[6:8]}"
            elif len(date_)==6:
                return f"14{date_[:2]}-{date_[2:4]}-{date_[4:6]}"
            else:
                print("bad data!")
                return None
        def clean_ua(x):
            return x.replace("اختيارخ ", "").replace('اختيارف ' ,"")  
        
        def day_to_ex(ex_date):
            y, m, d = map(int,(tuple(ex_date.split("-"))))
            return (jdt.date(y, m, d) - jdt.date.today()).days
        

        df[["ua", "strike_price","ex_date"]] = df.name.str.split("-", expand= True)
        df = df.assign(ua = df.ua.map(clean_ua))
        df = df.merge(ua_df, on="ua", how="inner")
        df = df[~df.ex_date.isnull()]
        df = df.assign(ex_date = df.ex_date.map(clean_date))
        df = df.assign(t = df.ex_date.map(day_to_ex))
        return df
    df = expand_option_info(mw_df)
    df = df.apply(pd.to_numeric, errors='ignore')
    return df

In [91]:
def query_df(cols, table):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"SELECT {cols} FROM {table}")
            cur.fetchone()
            conn.commit()
            return pd.DataFrame([i[0] for i in cur], columns=re.findall("[a-z A-Z]+", cols))
sigma_df = query_df("(ua,sigma)", "sigma")

In [92]:
df = option_data()
def option_type(x):
    return "call" if x.startswith("ض") else "put"
df = df.assign(type=df.symbol.map(option_type))
df = df.merge(sigma_df, on="ua", how="inner")
df["bs"] = df.apply(lambda x: round(bs.black_scholes(s_0=x["ua_final"], k=x["strike_price"], t=x["t"],sigma=float(x["sigma"]),type_=x["type"]),0),axis=1)
call = df[df.type =="call"]
put = df[df.type =="put"]


In [93]:
def married_put(df):
    """
    خرید داراییِ پایه و خریدِ همزمان اختیارِ فروش
    سود-زیان  برابرِ اختلافِ ارزشِ روز دارایی (در صورت کاهش تا حد قیمتِ اعمال) با ارزشِ خرید دارایی منهایِ ارزش خرید اختیارِ فروش است.
    :param df: with (ua, symbol, strike_price, ua_sell_price, sell_price) columns. ua: Underlying Asset
    :return: max_pot_profit: حداکثر سود بالقوه- در شرایطی ایجاد می‌شود که قیمتِ داراییِ پایه در تاریخ سر-رسید از قیمتِ اعمال بالاتر-برابر باشد
            break_even : اگر قیمتِ داراییِ پایه از این قیمت پایین-تر بیاد، معامله ،وارد زیان می‌شود
            pct_max_profit :
            current_profit : بر اساسِ قیمتِ کنونیِ داراییِ پایه محاسبه می‌شود
            pct_current_profit :
    """
    df = df[df.sell_price>0]
    stg = namedtuple("MarriedPut", "buy buy_")
    df["strategy"] = df.apply(
        lambda x: stg(buy=x["ua"], buy_=x["symbol"]), axis=1
    )
    df = df.assign(max_pot_profit="indefinite")
    df = df.assign(break_even=df.ua_sell_price + df.sell_price)
    df["current_profit"] = df.apply(
        lambda x: max(x["strike_price"], x["ua_sell_price"]) - x["ua_sell_price"] - x["sell_price"], axis=1)
    df = df.assign(pct_current_profit=df.current_profit / df.break_even * 100)
    df = df.assign(pct_profit_per_day=df.pct_current_profit / df.t)

    return df.sort_values(by="pct_profit_per_day", ascending=False)

In [94]:
def bear_call_spread(df):
        """
        فروش اختیار خرید با قیمتِ اعمالِ پایین-تر و خرید اختیارِ خریدِ با قیمتِ اعمالِ بالا-تر و تاریخِ اعمالِ همسان
        بیشینه سود برابر است باپرمیوم دریافتی
        بیشینه ضرر هم برابر است با تفاوتِ بین دو قیمتِ اعمال و پرمیومِ دریافتی
         :param df: with (ua, symbol, t, strike_price, ua_final, sell_price, buy_price) columns. ua: Underlying Asset
         :return:max_pot_loss: max_pot_loss
                max_pot_profit: max_pot_profit
                current_profit: current_profit
        """
        stg = namedtuple("BearCallSpread", "sell buy")
        groups = df.groupby(by=["ua", "t"])
        df_ = pd.DataFrame()
        for name, group in groups:
            group.reset_index(inplace=True)
            group = group.sort_values(by=["strike_price"], ascending=True)
            if len(group) > 1:
                combo_symbol = list(stg(sell=s, buy=b) for s, b in combinations(group.symbol, 2))
                combo_strike_price = list(stg(sell=s, buy=b) for s, b in combinations(group.strike_price, 2))
                combo_ob_price = list(stg(sell=s, buy=b) for s, b in combinations(group[["sell_price", "buy_price"]].itertuples(index=False), 2))
                combo_bs = list(stg(sell=s, buy=b) for s, b in combinations(group.bs, 2))
                max_pot_profit = [ps.buy_price - pb.sell_price for ps, pb in combo_ob_price]
                max_pot_loss = list(
                    map(add, [ss - sb for ss, sb in combo_strike_price], max_pot_profit)
                )
                current_profit = []
                for i in range(len(combo_strike_price)):
                    if all(
                        s >= group.ua_final.values[0] for s in combo_strike_price[i]
                    ):
                        current_profit.append(max_pot_profit[i])
                    elif all(
                        s <= group.ua_final.values[0] for s in combo_strike_price[i]
                    ):
                        current_profit.append(max_pot_loss[i])
                    else:
                        current_profit.append(
                            - group.ua_final.values[0]
                            + combo_strike_price[i].sell
                            + max_pot_profit[i]
                        )

                df_group = pd.DataFrame(
                    {
                        "symbol": combo_symbol,
                        "strike_price": combo_strike_price,
                        "t": group.t.values[0],
                        "bs": combo_bs,
                        "ua_final": group.ua_final.values[0],
                        "ob_price": combo_ob_price,
                        "max_pot_loss": max_pot_loss,
                        "max_pot_profit": max_pot_profit,
                        "current_profit": current_profit,
                    }
                )
                df_ = pd.concat([df_, df_group])
        return df_

In [131]:
def same_strike_and_ex_date_on_call_put(call, put):
    cols = [i for i in call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
    def cols_(x):
        if (x.endswith("_x")) and (x.startswith("ua_")):
            return x.replace("_x","")
        elif (x.endswith("_x")):
            return "call_" + x.replace("_x","")
        elif (x.endswith("_y")):
            return "put_" + x.replace("_y","")
        else:
            return x
    df = call.merge(put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
    df.columns = list(map(cols_, df.columns))
    return df


In [132]:
df_same_info_call_put = same_strike_and_ex_date_on_call_put(call=call[call.ob_depth==1], put=put[put.ob_depth==1])
df_same_info_call_put

,call_symbol,call_name,call_final,call_value,call_ob_depth,call_sell_vol,call_sell_price,call_buy_price,call_buy_vol,dt,...,put_symbol,put_name,put_final,put_value,put_ob_depth,put_sell_vol,put_sell_price,put_buy_price,put_buy_vol,put_bs
0,ضخود3061,اختيارخ خودرو-1600-1402/03/03,1930,235428000,1,1,2199,1902,12,1402-01-02 20:29:38,...,طخود3061,اختيارف خودرو-1600-1402/03/03,26,0,1,1,45,0,0,0.0
1,ضخود0109,اختيارخ خودرو-3000-1402/01/09,422,1548569000,1,292,430,405,199,1402-01-02 20:29:38,...,طخود0109,اختيارف خودرو-3000-1402/01/09,30,0,1,50,70,0,0,0.0
2,ضستا6002,اختيارخ شستا-665-1402/06/08,786,272039000,1,50,774,750,10,1402-01-02 20:29:38,...,طستا6002,اختيارف شستا-665-1402/06/08,5,0,1,0,0,1,100,0.0
3,ضستا6010,اختيارخ شستا-1465-1402/06/08,298,105223689000,1,1,280,270,325,1402-01-02 20:29:38,...,طستا6010,اختيارف شستا-1465-1402/06/08,144,899372000,1,559,150,140,1000,116.0
4,ضستا2011,اختيارخ شستا-800-1402/02/13,580,0,1,130,800,550,1000,1402-01-02 20:29:38,...,طستا2011,اختيارف شستا-800-1402/02/13,11,0,1,0,0,4,20,0.0
5,ضستا6008,اختيارخ شستا-1265-1402/06/08,401,400750000,1,3,414,380,741,1402-01-02 20:29:38,...,طستا6008,اختيارف شستا-1265-1402/06/08,66,0,1,100,200,68,6,41.0
6,ضستا6006,اختيارخ شستا-1065-1402/06/08,479,3572328000,1,582,480,472,319,1402-01-02 20:29:38,...,طستا6006,اختيارف شستا-1065-1402/06/08,34,2154572000,1,1249,36,33,1500,9.0
7,ضستا2012,اختيارخ شستا-900-1402/02/13,448,37243368000,1,0,0,454,20,1402-01-02 20:29:38,...,طستا2012,اختيارف شستا-900-1402/02/13,6,141328000,1,3291,6,5,1450,0.0
8,ضستا6004,اختيارخ شستا-865-1402/06/08,625,230761427000,1,816,614,610,20,1402-01-02 20:29:38,...,طستا6004,اختيارف شستا-865-1402/06/08,14,37603000,1,3329,14,13,128,1.0
9,ضستا6009,اختيارخ شستا-1365-1402/06/08,337,903121000,1,100,373,315,512,1402-01-02 20:29:38,...,طستا6009,اختيارف شستا-1365-1402/06/08,210,0,1,0,0,127,2,73.0


In [143]:
def long_straddle(df):
    stg = namedtuple("LongStraddle", "buy_c buy_p")
    df = df.assign(max_pot_loss = - df.call_sell_price - df.put_sell_price)
    df = df.assign(lower_break_even = df.strike_price + df.max_pot_loss,
                   upper_break_even = df.strike_price - df.max_pot_loss)
    df["current_profit"] = df.apply(lambda x: max(x["max_pot_loss"], abs(x["ua_final"] - x["strike_price"]) + x["max_pot_loss"]), axis=1)
    return df.sort_values(by="current_profit", ascending=False)

def short_straddle(df):
    stg = namedtuple("ShortStraddle", "sell_c sell_p")
    df = df.assign(max_pot_profit =  df.call_buy_price + df.put_buy_price)
    df = df.assign(lower_break_even = df.strike_price - df.max_pot_profit,
                   upper_break_even = df.strike_price + df.max_pot_profit)
    df["current_profit"] = df.apply(lambda x: min(x["max_pot_profit"], - abs(x["ua_final"] - x["strike_price"]) + x["max_pot_profit"]), axis=1)
    return df.sort_values(by="current_profit", ascending=False)
    

In [116]:
df_bsc = bear_call_spread(call[(call.ob_depth==1) &(call.sell_price>0) &(call.buy_price>0)])

In [144]:
df_long_straddle = long_straddle(df_same_info_call_put[(df_same_info_call_put.call_sell_price > 0) & (df_same_info_call_put.put_sell_price > 0)])
df_short_straddle= short_straddle(df_same_info_call_put[(df_same_info_call_put.call_buy_price > 0) & (df_same_info_call_put.put_buy_price > 0)])

In [134]:
df_long_straddle.to_excel("long_straddle.xlsx")

In [146]:
df_long_straddle

,call_symbol,call_name,call_final,call_value,call_ob_depth,call_sell_vol,call_sell_price,call_buy_price,call_buy_vol,dt,...,put_ob_depth,put_sell_vol,put_sell_price,put_buy_price,put_buy_vol,put_bs,max_pot_loss,lower_break_even,upper_break_even,current_profit
10,ضستا0105,اختيارخ شستا-1100-1402/01/09,221,11096928000,1,880,210,201,599,1402-01-02 20:29:38,...,1,500,10,0,0,0.0,-220,880,1320,-26
1,ضخود0109,اختيارخ خودرو-3000-1402/01/09,422,1548569000,1,292,430,405,199,1402-01-02 20:29:38,...,1,50,70,0,0,0.0,-500,2500,3500,-124
8,ضستا6004,اختيارخ شستا-865-1402/06/08,625,230761427000,1,816,614,610,20,1402-01-02 20:29:38,...,1,3329,14,13,128,1.0,-628,237,1493,-199
3,ضستا6010,اختيارخ شستا-1465-1402/06/08,298,105223689000,1,1,280,270,325,1402-01-02 20:29:38,...,1,559,150,140,1000,116.0,-430,1035,1895,-259
6,ضستا6006,اختيارخ شستا-1065-1402/06/08,479,3572328000,1,582,480,472,319,1402-01-02 20:29:38,...,1,1249,36,33,1500,9.0,-516,549,1581,-287
12,ضصاد3020,اختيارخ وبصادر-1400-1402/03/17,706,0,1,50,850,1,1000,1402-01-02 20:29:38,...,1,12,200,5,12,0.0,-1050,350,2450,-425
0,ضخود3061,اختيارخ خودرو-1600-1402/03/03,1930,235428000,1,1,2199,1902,12,1402-01-02 20:29:38,...,1,1,45,0,0,0.0,-2244,-644,3844,-468
5,ضستا6008,اختيارخ شستا-1265-1402/06/08,401,400750000,1,3,414,380,741,1402-01-02 20:29:38,...,1,100,200,68,6,41.0,-614,651,1879,-585
19,ضسمگا703,اختيارخ سمگا-8500-14020709,940,0,1,51,2002,333,1,1402-01-02 20:29:38,...,1,3,950,600,7,1295.0,-2952,5548,11452,-1182


In [145]:
df_short_straddle

,call_symbol,call_name,call_final,call_value,call_ob_depth,call_sell_vol,call_sell_price,call_buy_price,call_buy_vol,dt,...,put_ob_depth,put_sell_vol,put_sell_price,put_buy_price,put_buy_vol,put_bs,max_pot_profit,lower_break_even,upper_break_even,current_profit
16,ضبساما305,اختيارخ بساما-12000-14020310,2187,196839000,1,10,2299,1905,12,1402-01-02 20:29:38,...,1,0,0,222,1,212.0,2127,9873,14127,1727
23,ضترا3024,اختيارخ شتران-4000-1402/03/28,1279,1503910000,1,15,1345,1263,11,1402-01-02 20:29:38,...,1,0,0,10,12,40.0,1273,2727,5273,678
5,ضستا6008,اختيارخ شستا-1265-1402/06/08,401,400750000,1,3,414,380,741,1402-01-02 20:29:38,...,1,100,200,68,6,41.0,448,817,1713,419
9,ضستا6009,اختيارخ شستا-1365-1402/06/08,337,903121000,1,100,373,315,512,1402-01-02 20:29:38,...,1,0,0,127,2,73.0,442,923,1807,371
6,ضستا6006,اختيارخ شستا-1065-1402/06/08,479,3572328000,1,582,480,472,319,1402-01-02 20:29:38,...,1,1249,36,33,1500,9.0,505,560,1570,276
3,ضستا6010,اختيارخ شستا-1465-1402/06/08,298,105223689000,1,1,280,270,325,1402-01-02 20:29:38,...,1,559,150,140,1000,116.0,410,1055,1875,239
8,ضستا6004,اختيارخ شستا-865-1402/06/08,625,230761427000,1,816,614,610,20,1402-01-02 20:29:38,...,1,3329,14,13,128,1.0,623,242,1488,194
13,ضستر2010,اختيارخ خگستر-5500-1402/02/24,400,6000000,1,0,0,222,1,1402-01-02 20:29:38,...,1,0,0,850,1,617.0,1072,4428,6572,172
2,ضستا6002,اختيارخ شستا-665-1402/06/08,786,272039000,1,50,774,750,10,1402-01-02 20:29:38,...,1,0,0,1,100,0.0,751,-86,1416,122
7,ضستا2012,اختيارخ شستا-900-1402/02/13,448,37243368000,1,0,0,454,20,1402-01-02 20:29:38,...,1,3291,6,5,1450,0.0,459,441,1359,65
